In [ ]:
%load show_original_decoded.py

In [21]:
# %load ImageOp.py
class ImageOp:
    def __init__(self):        
        self.hits = 0
        self.miss = 0

    def get_cache_value(self, img):
        import hashlib
        hasher = hashlib.md5()
        hasher.update(str(img.shape).encode('utf-8'))
        hasher.update(img.data)
        for_hash = hasher.hexdigest()    
        if for_hash in cache:
            self.hits = self.hits + 1
            return for_hash, cache[for_hash]
        self.miss = self.miss + 1
        return for_hash, None
    
class HistoOp(ImageOp):    
    def __call__(self, img):
        print('HistoOp {}'.format(img))

In [19]:
import numpy as np
op = HistoOp()
op(np.array([]))

HistoOp []


In [ ]:
cache = {}

In [ ]:
hits, miss = 0, 0
def get_cache_value(img):
    global hits, miss
    import hashlib
    hasher = hashlib.md5()
    hasher.update(str(img.shape).encode('utf-8'))
    hasher.update(img.data)
    for_hash = hasher.hexdigest()    
    if for_hash in cache:
        hits = hits + 1
        return for_hash, cache[for_hash]
    miss = miss + 1
    return for_hash, None

print(len(cache))

In [ ]:
from keras_preprocessing.image import ImageDataGenerator
from skimage import exposure, img_as_ubyte
import numpy as np

def whiten_img(img):
    imin, imax = np.min(img), np.max(img)
    img_ubyte = img_as_ubyte((img - imin)/(imax - imin))
    return img_ubyte
    
def filter_shift_img(img, shift_model=None, use_clahe=False):
    img_ubyte = whiten_img(img)

    # save original shape to restore upon return
    orig_shape = img.shape
    img_ubyte = np.squeeze(img_ubyte)
    
    # check the cache
    for_hash, final_img = get_cache_value(img_ubyte)
    if not(final_img is None):
        return final_img
    
    # apply clahe
    if use_clahe:
        filtered_img = \
            exposure.equalize_adapthist(img_ubyte,
                kernel_size=(8,8), clip_limit=0.03)
    else:
        filtered_img = img
        
    # apply shift
    if shifter_model != None:
        latent = shifter_model.predict(clahed_img)        
        print(latent)

    final_img = np.reshape(filtered_img, orig_shape)
    cache[for_hash] = final_img
        
    return final_img

train_datagen = ImageDataGenerator(
    horizontal_flip=False,
    preprocessing_function=lambda img:filter_shift_img(img,use_clahe=True))

In [ ]:
sz = 128
nihcc_x8 = r'G:/DataAll/chest-nihcc'
train_generator = train_datagen.flow_from_directory(
        '{}/by_class'.format(nihcc_x8),
        color_mode='grayscale',
        target_size=(sz, sz),
        batch_size=128,
        class_mode='input',
        classes=['no_finding'])
train10, match10 = next(train_generator)
print(train10.shape, match10.shape)
show_original_decoded(train10, match10, sz)